# Package and definitions


In [1]:
import requests as req
import pandas as pd
import os
import re
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try :
  from termcolor import colored
except :
  !pip install termcolor
  from termcolor import colored
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
try :
  import transformers
except :
  !pip install transformers
  import transformers
from transformers import RobertaForMaskedLM, BertTokenizerFast, RobertaTokenizer, AutoModel, AutoModelForMaskedLM, AutoTokenizer, RobertaConfig, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.utils.class_weight import compute_class_weight
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from itertools import compress
import seaborn as sns
from tqdm import *
import time
! pip install datasets


# torch.cuda.is_available() returns a boolean to check if the GPU can be used or not
if torch.cuda.is_available():
  # if CUDA is available set 'cuda' as the device
  device = 'cuda'
  # and then print the name of the GPU
  print('DEVICE = ', colored(torch.cuda.get_device_name(0), "green" ) )
else:
  # else, set 'cpu' as device
  device = 'cpu'
  # just print than the CPU is used. Alternatively you can check your CPU with the following command (linux based) in the next cell:
  # ! lscpu
  print('DEVICE = ', colored('CPU', "blue"))



git_url = "https://raw.githubusercontent.com/chlolv/NLP_Project/bert/"
t0_url = "text_0.txt"
t1_url = "text_1.txt"
t2_url = "text_2.txt"


     |████████████████████████████████| 4.0 MB 2.9 MB/s 
     |████████████████████████████████| 895 kB 24.3 MB/s 
     |████████████████████████████████| 6.6 MB 12.6 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 42.0 MB/s 
     |████████████████████████████████| 212 kB 46.7 MB/s 
     |████████████████████████████████| 136 kB 49.4 MB/s 
     |████████████████████████████████| 127 kB 51.5 MB/s 
     |██

DEVICE =  Tesla K80


### Pipeline to prepare data for prediction

In [2]:
t0 = req.get(git_url + t0_url)
t0 = t0.text

lines = t0.split('\n')

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', max_len=512, bos_token = '<s>', eos_token = '</s>',
                                             sep_token = '</s>', cls_token = '<s>', unk_token = '<unk>',
                                             pad_token = '<pad>', mask_token = '<mask>')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [4]:
new_names = ['Harry', 'Potter', 'Ron', 'Weasley','Hermione', 'Granger','Snape', 'Severus',
              'Albus', 'Dumbledore','Dursley', 'Vernon', 'Dudley', 'Petunia','Draco', 'Malfoy']
num_added_tokens = tokenizer.add_tokens(new_names)

new_tokens = [tokenizer(token)['input_ids'][1] for token in new_names]
special_tokens = [tokenizer(token)['input_ids'][1] for token in ["<s>","</s>","<pad>","<unk>","<mask>"]]


In [5]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)

In [6]:
labels = torch.tensor([x for x in batch.input_ids])
mask = torch.tensor([x for x in batch.attention_mask])

# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
mask_arr = (rand < .75) * np.prod([input_ids == ele for ele in new_tokens])
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = special_tokens[4]  # our custom [MASK] token == 3

encodings = {'input_ids': input_ids, 'attention_mask': mask,
             'labels': labels}

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}


In [8]:
dataset = Dataset(encodings)
loader = torch.utils.data.DataLoader(dataset, batch_size=12, shuffle=True)

### Training the model

In [11]:
config = RobertaConfig(
    vocab_size=len(tokenizer),  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    # type_vocab_size=1,
    bos_token = '<s>', 
    eos_token = '</s>',
    sep_token = '</s>', 
    cls_token = '<s>', 
    unk_token = '<unk>',
    pad_token = '<pad>', 
    mask_token = '<mask>'
)

model = RobertaForMaskedLM.from_pretrained('roberta-base', do_basic_tokenize=False)
vocab = model.get_vocab()
for new_word in new_names :
  vocab.update({new_word : len(vocab)})

#model = RobertaForMaskedLM.from_pretrained('roberta-base', vocab_file = vocab, do_basic_tokenize=False)

TypeError: ignored

In [10]:
model.update

AttributeError: ignored

In [ ]:
model = model(config)

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
len(tokenizer)

In [ ]:
model.train()
optim = AdamW(model.parameters(), lr=1e-4)

epochs = 2

for epoch in range(epochs):
  # setup loop with TQDM and dataloader
  loop = tqdm(loader, leave=True)
  for batch in loop:
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # pull all tensor batches required for training
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    # process
    outputs = model(input_ids, 
                    attention_mask=attention_mask, 
                    labels = labels)
    # extract loss
    loss = outputs.loss
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()
    # print relevant info to progress bar
    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())

In [16]:
len(attention_mask)

12

### Testing the model on names 

In [ ]:
# need to feed the model a text with [MASK] labels

In [ ]:
model.save_pretrained('pretrained_harrypotbert3')

In [ ]:
from transformers import pipeline

In [ ]:
fill = pipeline('fill-mask', model='pretrained_harrypotbert3', tokenizer='harrypotbert')

In [ ]:
fill(f"Ron held {fill.tokenizer.mask_token}'s hand.")

In [ ]:
fill(f"Harry was happy to see {fill.tokenizer.mask_token} in the castle.")